In [26]:
import json
import re

# 输入／输出文件路径
in_path = '../data/Roads/overpass/export (9).geojson'
out_path = '../data/Roads/overpass/9export_converted.geojson'

In [27]:
# 正则：匹配数字 + “mph” （忽略大小写）
pattern = re.compile(r'(\d+(?:\.\d+)?)\s*mph', re.IGNORECASE)

# 读入原 GeoJSON
with open(in_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

count = 0
for feat in data.get('features', []):
    props = feat.get('properties', {})
    ms = props.get('maxspeed', '')
    m = pattern.match(ms)
    if m:
        mph = float(m.group(1))
        kmh = mph * 1.60934
        new_val = f"{round(kmh)}"
        props['maxspeed'] = new_val
        count += 1

# 写出到新文件
with open(out_path, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"共转换了 {count} 条 maxspeed，输出保存到 {out_path}")


共转换了 3362 条 maxspeed，输出保存到 ../data/Roads/overpass/9export_converted.geojson
